In [5]:
import tensorflow as tf
from tensorflow.keras import layers
import yaml

# Define your custom model architecture
def build_model(height, width, channels, num_landmarks):
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_landmarks * 2)  # Assuming num_landmarks landmarks with (x, y) coordinates
    ])
    return model

# Load YAML configuration file
with open("/home/selvatharrun/Documents/GitHub/cephalometric-analysis/face cephalometry.v6i.yolov8/data.yaml", "r") as f:
    config = yaml.safe_load(f)

# Extract configuration parameters
train_dir = config["train"]
val_dir = config["val"]
test_dir = config["test"]
nc = config["nc"]
names = config["names"]

# Assuming you have height, width, channels, and num_landmarks defined somewhere
# Height, width, and channels are dimensions of your input images
# num_landmarks is the number of landmarks you want to detect
height=640
width=640
channels=3
num_landmarks=26
# Build your model
model = build_model(height, width, channels, num_landmarks)

# Compile your model
model.compile(optimizer='adam', loss='mean_squared_error')

# Define data loading and augmentation pipelines (not provided here)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data augmentation parameters
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Define batch size
batch_size = 32

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='other'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='other'
)

# Define data augmentation parameters for test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='other'
)

# Train your model
# Assuming you have prepared your dataset and data generators
# train_generator = ...
# val_generator = ...
# test_generator = ...
history = model.fit(train_generator, epochs=10, validation_data=val_generator)

# Evaluate your model on the test set
test_loss = model.evaluate(test_generator)
print("Test Loss:", test_loss)

# Save your trained model
model.save("your_model.h5")


ValueError: Invalid class_mode: other; expected one of: {'binary', 'sparse', 'input', None, 'categorical'}

In [6]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import yaml

def build_model(height, width, channels, num_landmarks):
    # Adjust final layer to output 26 x and 26 y coordinates for all landmarks
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        # Final layer with 52 units (26 landmarks * 2 coordinates per landmark)
        layers.Dense(num_landmarks * 2)
    ])
    return model


# Load YAML configuration file
with open("/home/selvatharrun/Documents/GitHub/cephalometric-analysis/face cephalometry.v6i.yolov8/data.yaml", "r") as f:
    config = yaml.safe_load(f)

# Extract configuration parameters
train_dir = config["train"]
val_dir = config["val"]
test_dir = config["test"]
nc = config["nc"]
names = config["names"]

# Assuming you have height, width, channels, and num_landmarks defined somewhere
# Height, width, and channels are dimensions of your input images
# num_landmarks is the number of landmarks you want to detect (set to 26 for all landmarks)
height = 640
width = 640
channels = 3
num_landmarks = 26

# Build your model with the modified final layer
model = build_model(height, width, channels, num_landmarks)


# Compile your model (assuming mean squared error loss is suitable for your task)
model.compile(optimizer='adam', loss='mean_squared_error')

# Define data augmentation parameters (adjust as needed)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

# Define batch size
batch_size = 32

# Create data generators (ensure class_mode is set to None for regression)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode=None,  # No class mode for regression
    classes=[] # Automatically infer classes from directory structure
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode=None,  # No class mode for regression
    classes=[]  # Automatically infer classes from directory structure
)

# Assuming you have a list 'landmark_data' containing ground truth landmark coordinates for each image

# Extract target data for a single image (modify based on your data structure)
image, landmarks = next(train_generator)  # Assuming generator yields (image, landmarks)
target_landmarks = np.array(landmarks).flatten()  # Flatten the landmark coordinates (x, y for each landmark)

# Assuming you have a list 'train_targets' to store target data for all training images
train_targets.append(target_landmarks)

# Similar steps for validation and test targets


test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode=None,  # No class mode for regression
    classes=None  # Automatically infer classes from directory structure
)

# Train your model
history = model.fit(train_generator, epochs=10, validation_data=val_generator)

# Evaluate your model on the test set
test_loss = model.evaluate(test_generator)
print("Test Loss:", test_loss)

# Save your trained model
model.save("your_model.h5")


Found 297 images belonging to 2 classes.
Found 82 images belonging to 2 classes.
Found 38 images belonging to 2 classes.
Epoch 1/10


ValueError: in user code:

    File "/home/selvatharrun/.local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/selvatharrun/.local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/selvatharrun/.local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/selvatharrun/.local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1152, in train_step
        self._validate_target_and_loss(y, loss)
    File "/home/selvatharrun/.local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1106, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing. Your model was compiled with loss=mean_squared_error, and theref

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import yaml


def build_model(height, width, channels, num_landmarks):
    # Adjust final layer to output 26 x and 26 y coordinates for all landmarks
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        # Final layer with 52 units (26 landmarks * 2 coordinates per landmark)
        layers.Dense(num_landmarks * 2)
    ])
    return model


# Load YAML configuration file
with open("/home/selvatharrun/Documents/GitHub/cephalometric-analysis/face cephalometry.v6i.yolov8/data.yaml", "r") as f:
    config = yaml.safe_load(f)

# Extract configuration parameters
train_dir = config["train"]
val_dir = config["val"]
test_dir = config["test"]
nc = config["nc"]
names = config["names"]

# Assuming you have height, width, channels, and num_landmarks defined somewhere
# Height, width, and channels are dimensions of your input images
# num_landmarks is the number of landmarks you want to detect (set to 26 for all landmarks)
height = 640
width = 640
channels = 3
num_landmarks = 26

# Build your model with the modified final layer
model = build_model(height, width, channels, num_landmarks)


# Compile your model (assuming mean squared error loss is suitable for your task)
model.compile(optimizer='adam', loss='mean_squared_error')

# Define data augmentation parameters (adjust as needed)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

# Define batch size
batch_size = 32

# Create data generators (ensure class_mode is set to None for regression)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode=None,  # No class mode for regression
    classes=[]  # Automatically infer classes from directory structure
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode=None,  # No class mode for regression
    classes=[]  # Automatically infer classes from directory structure
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode=None,  # No class mode for regression
    classes=None  # Automatically infer classes from directory structure
)

# List to store training target landmark coordinates
train_targets = []

# Load landmark data for training (replace with your implementation)
for image_path in train_generator.filenames:
  # Extract image ID from filepath (modify based on your filename structure)
  image_id
